In [1]:
import os
import sys

#Auto update imports when python files in src is updated
%load_ext autoreload
%autoreload 2
import sys

from src.utils import get_disp_eiendommer, convert_crs, boundingBox, to_rgb
from kornmo.sentinel.sentinel_helpers import download_timeseries_from_bbox
from kornmo.sentinel.sentinel_evalscripts import natural_color, all_bands
from kornmo.sentinel.storage import SentinelDataset
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

import warnings
warnings.filterwarnings("ignore")

data_location = "../../../kornmo-data-files/raw-data"

ModuleNotFoundError: No module named 'sentinel'

In [2]:
coordinates = pd.read_csv(os.path.join(data_location, 'farm-information/old farm location information/centroid_coordinates_new.csv'))
dataset = pd.read_csv(os.path.join(data_location, '../dataset_filtered-and-normalized.csv'))
disp_properties = get_disp_eiendommer()

disp_properties.head()

Reading 'disponerte_eiendommer.gpkg'...


,orgnr,year,geometry
0,969102404,2017,"MULTIPOLYGON (((199287.149 6584342.538, 199295..."
1,983375782,2017,"MULTIPOLYGON (((284208.764 6624834.683, 284208..."
2,969924358,2017,"MULTIPOLYGON (((284560.510 6687767.290, 284558..."
4,869249572,2019,"MULTIPOLYGON (((170263.991 6598866.505, 170261..."
5,887932522,2019,"MULTIPOLYGON (((263269.940 7024098.969, 263265..."


In [14]:
print(len(list(set(disp_properties['orgnr'].tolist()))))
print(len(list(disp_properties['orgnr'])))
row = disp_properties.loc[disp_properties['orgnr'] == str(orgnrs[i])].sort_values(by=['year']).iloc[0]


10154
13123


KeyError: 983375782

In [3]:
def append_orgnr(orgnr):
    file = open('download_progress.txt', 'a')
    file.write(f"{str(orgnr)},")
    file.close()

def get_done_orgnrs():
    file = open('download_progress.txt', 'r')
    nrs = file.read().split(',')[:-1]
    file.close()
    return nrs

In [5]:
years = [2020]
sd = SentinelDataset('E:/MasterThesisData/Satellite_Images/sentinel_100x100_new_data.h5', create_if_missing=True)
orgnrs = [str(orgnr) for orgnr in list(set(dataset['orgnr'].tolist()))]
disp_orgnrs = list(set(disp_properties['orgnr'].tolist()))

for i in tqdm(range(0, len(orgnrs)), desc=f'Prosessing images...'):
    done_orgnrs = get_done_orgnrs()
    #print(len(done_orgnrs))
    if (orgnrs[i] in disp_orgnrs) and (orgnrs[i] not in done_orgnrs):
        row = disp_properties.loc[disp_properties['orgnr'] == orgnrs[i]].sort_values(by=['year']).iloc[0]
        point = convert_crs([row['geometry'].centroid])[0]
        bbox = boundingBox(point.y, point.x, 1)
        for year in years:
            imgs, _ = download_timeseries_from_bbox(bbox, (year, 3, 1), (year, 10, 1), 30, evalscript=all_bands)
            sd.store_images(imgs, row['orgnr'], year)
        append_orgnr(orgnrs[i])

    else:
        #print(f"Skipping {orgnrs[i]}")
        ...


Prosessing images...:  21%|██        | 2151/10220 [1:30:08<5:38:09,  2.51s/it] 


KeyboardInterrupt: 

In [67]:
from sentinel.prev_utils import to_rgb
sd = SentinelDataset('E:/MasterThesisData/Satellite_Images/sentinel_100x100_new_data.h5')
orgnrs = list(set(disp_properties['orgnr'].tolist()))

#print(disp_properties.loc[disp_properties['orgnr'] == orgnrs[200]].sort_values(by=['year']).iloc[0])
sd.contains(981893131, 2020)
# data = sd.get_images(969113600, 2021)
# data = [to_rgb(img) for img in data]
# ncols = 5
# nrows = 6
# aspect_ratio = 1
# subplot_kw = {'xticks': [], 'yticks': [], 'frame_on': False}
# fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows), subplot_kw=subplot_kw)
#
# for idx, image in enumerate(data):
#     ax = axs[idx // ncols][idx % ncols]
#     ax.imshow(np.clip(image, 0, 1))
#     #ax.set_title(f'{slots[idx][0]}  -  {slots[idx][1]}', fontsize=10)
#
# plt.tight_layout()


True

In [4]:
import h5py

with h5py.File('E:/MasterThesisData/Satellite_Images/sentinel_100x100_new_data.h5') as f:
    #print(f['images'].keys())
    print(len(f['images'].keys()))

    # file = open('download_progress.txt', 'w')
    # for nr in list(f['images'].keys()):
    #     file.write(f'{str(nr)},')
    # file.close()


f = open('download_progress.txt', 'r')
text = f.read()
print(len(text.split(',')[:-1]))
print(text.split(',')[-2])
f.close()

1524
1524
969609347
